In [1]:
import requests
from tqdm import tqdm
import numpy as np
import pandas as pd
import os.path
from time import sleep


In [11]:
url_repo = 'https://api.github.com/repos/facebook/react/contributors?per_page=50'
authority = ('login', 'password') # basic authentification for enlarging API limits
r = requests.get(url_repo, auth=authority)

In [12]:
top50contributors = defaultdict(list)
for contributor in r.json():
    top50contributors['login'].append(contributor['login'])
    top50contributors['contributions'].append(contributor['contributions'])

df = pd.DataFrame(top50contributors)
df.to_csv(os.path.join(*['data','contributors_info.csv']))

In [13]:
def user_files_data(user_data):
    url_template = 'https://api.github.com/repos/facebook/react/commits?per_page=100&page={page}&author={login}'
    login, contributions = user_data
    contributor_files = {'login':login, 'files':set()}
    pages_count = contributions // 100 + (1 if contributions % 100 > 0 else 0)
    for page in tqdm(range(1, pages_count + 1)):
        url = url_template.format(login=login, page=page)
        r = requests.get(url, auth=authority)
        for commit in r.json():
            try:
                commit_url = commit['commit']['url'].replace('react/git/commits', 'react/commits')
                commit_page = requests.get(commit_url, auth=authority)
                files = commit_page.json()['files']
                for file in files:
                    contributor_files['files'].add(file['filename'])
                top50contributors['files'].append(contributor_files)
            except Exception:
                return 
    txt = open(os.path.isfile(os.path.join(*['users', login + '.txt'])), 'w')
    txt.write(contributor_files['login'] + '\n')
    for filename in contributor_files['files']:
        txt.write(filename+'\n')
    txt.close()

In [15]:
df = pd.read_csv(os.path.join(*['data','contributors_info.csv']), index_col=0)
user_data = [tuple(x) for x in df.to_numpy()]
num = sum([1 if os.path.isfile(os.path.join(*['users', users[0] + '.txt'])) else 0 for users in user_data])


while num != 50:
    print('Remaining:', 50 - num)
    for users in user_data:
        if os.path.isfile(os.path.join(*['users', users[0] + '.txt'])):
            continue
        print(users)
        try:
            user_files_data(users)
        except Exception:
            print('An error has occured while downloading: reach API limits.')
            sleep(600)
            break
        num += 1
else:
    print('Done')

Done
